# Importar Librerias

In [14]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
%matplotlib inline

import sys
import os
main_dir = os.path.abspath('..')
export_path = os.path.join(main_dir,'00_Utilities')
# adding Folder_2 to the system path
sys.path.insert(0, export_path)
from dirFunc import *

# Importar Datos

In [15]:
ruta_origen = 'Datos'

# De SAP
hOs = pd.read_csv(ruta_origen + '\os.csv')
hOperaciones = pd.read_csv(ruta_origen + '\operaciones.csv')
hHh = pd.read_csv(ruta_origen + '\hh.csv')
hRepuestos = pd.read_csv(ruta_origen + '\\repuestos.csv')
hWip = pd.read_csv(ruta_origen + '\wip.csv')

# De Qplus
hQplus = pd.read_csv(ruta_origen + '\qplus.csv')

# De TTA
hTta = pd.read_csv(ruta_origen + '\\tta.csv')

# De Trabajadores
hTrabajadores = pd.read_csv(ruta_origen + '\\trabajadores.csv')

# De Faena
# hFaena = pd.read_csv(ruta_origen + '\\faena.csv')

# Historial cursos
hCursos = pd.read_csv(ruta_origen + '\\cursos.csv')
hHistorial = pd.read_csv(ruta_origen + '\\historial.csv')
hCursosAll = pd.read_csv(ruta_origen + '\\cursosAll.csv')

# Limpieza de Datos

## QPlus

### QPlus global

In [16]:
# Limpiar Tabla
# Limpiar Columnas
aQplus = hQplus.copy().drop(columns=['ID','ID Oportunidad','Orden Compra','Nº de Componente','Canti dad','Nombre Componente','Serie','Garan tía','Código de Garantía', 'Fecha de Despacho','OS Liquidación','nroCliente Final', 'Cliente Final','Subestado','Fec Recep Doc','Column1','Fec Activación','Fec Informe Presup TTA','Fec Informe Repuesto','Fec Informe IR o AFA','Fecha Envío','Programar desde','F.Comprom Cot.','Nro Aviso','OS Anterior','Resol Final Garantía', 'Fecha Resolución']) 
# Limpiar Filas
aQplus = aQplus.dropna(subset=['S/N Componente']).reset_index(drop=True)

aQplus.columns = ['OS','cliente','tipoComponente','modelo','estado','fechaLlegada','fechaPrometida','fechaTermino','NParte','serieComp','serieEquipo','tipoOrden','tipoReparacion','horasComp']

In [17]:
# Aplicar Filtros
tipoComponente = ['CO','DI','MA','MF','MO','TR']
estado = ['DESPACHO','TERMINADO']
tipoReparacion = ['Solo Prueba','Cambio posicion','Solo AFA']

aQplus['estado'] = aQplus['estado'].replace({'53 - DESPACHADO A CLIENTE': 'DESPACHO',
                                             '23 - TERMINADO': 'TERMINADO'})
aQplus = aQplus.loc[aQplus['tipoComponente'].isin(tipoComponente)]
aQplus = aQplus.loc[aQplus['estado'].isin(estado)]
aQplus = aQplus.loc[~aQplus['tipoReparacion'].isin(tipoReparacion)]

# Solucionar valor de Horas Componente
horasComp = aQplus['horasComp'].apply(lambda x: x.replace('.','').replace(',','.'))
aQplus['horasComp'] = pd.to_numeric(horasComp)

### QPlus con filtros

In [18]:
fQplus = aQplus.copy()

# Filtro de alcance del proyecto
years = [2020,2021]
reparacion = ['Overhaul','Proceso Continuo']
cliente = "INTERCAMBIO"
sinSerie = ['SIN SERIE','SS','SIN INFORMACION','NO ENTREGADA','VER PLACA']
sinSerieEquipo = ['S/N'] # Viendo si se aplica este filtro.

fQplus = fQplus.dropna(subset=['fechaTermino','serieComp'])
fQplus = fQplus.dropna(subset=['cliente','tipoReparacion'])

fQplus['fechaTermino'] = pd.to_datetime(fQplus['fechaTermino'], errors='coerce')
fQplus = fQplus.loc[fQplus['fechaTermino'].dt.year.isin(years)]
fQplus = fQplus.loc[fQplus['tipoReparacion'].isin(reparacion)]
fQplus = fQplus.loc[~fQplus['serieComp'].isin(sinSerie)]
fQplus = fQplus[fQplus.cliente.str.contains(cliente)]

# Only consider numeric OS
fQplus = addOSMadreIndex(fQplus, dropOrden=True, ordenName = 'OS', osMadreName = 'OSMadre')
# # Optimizando
fQplus = fQplus.drop(columns=['cliente'])

## Repuestos

In [19]:
fRepuestos = hRepuestos.copy().drop(columns=['De período','Sociedad','Centro','Clase de coste','Moneda del informe','Ud. cantidad contab.','Número de personal','Documento compras','Nº docum.refer.','Usuario','Cta.contrapartida','Texto breve de material','Denom.clase de coste'])
# Renombrar columnas a otras mas descriptivas
fRepuestos.columns = ['fecha', 'Orden','Costo','Cantidad','Material'] # ,'NombreMaterial'
# Eliminar filas vacias
fRepuestos = fRepuestos.dropna(subset=['Material']).reset_index(drop=True)

# Formateando la orden
oslist = list(fQplus.index.values)
fRepuestos = addOSMadreIndex(fRepuestos, dropOrden=False, ordenName = 'Orden', osFilter=oslist,index=False)
fRepuestos = fRepuestos.groupby(['OSMadre','Material'])['Costo','Cantidad'].sum().reset_index()

## Trabajadores

In [20]:
fTrabajadores = hTrabajadores.copy()
fTrabajadores.drop(columns=['RUT','Nombre','Posición ','Ubicación','Ubicación Workday','Centro de costo','Representante RRHH','Nacionalidad','Sexo','Fecha antigüedad','Fecha retiro','Días Trabajados','Relación contrato'],inplace=True)
fTrabajadores.columns = ['fechaNacimiento','fechaIngreso','Nivel','NPersonal']
fTrabajadores.fechaNacimiento = pd.to_datetime(fTrabajadores['fechaNacimiento'])
fTrabajadores.fechaIngreso = pd.to_datetime(fTrabajadores['fechaIngreso'])

# Filtrar trabajadores por operarios mecanicos o tecnicos
niveles = {'M1':"1",'M2':'2','M3':'3','T1':'4','T2':'5','T3':'6','S1':'7'}
listaNiveles = list(niveles.keys())
fTrabajadores = fTrabajadores[fTrabajadores['Nivel'].isin(listaNiveles)]

## Mano de Obra o HH

In [21]:
fHh = hHh.copy()
fHh = fHh.drop(columns=['Centro (plan)','Clase orden','Pto.trab.(real)','Cl.activ.(real)','FechaInicioReal',
        'Hora inic.real','Fecha fin real','Hora fin real','Fecha contab.','Creado por','Creado el',
        'Notific.final','Notificación','Status sistema','Equipo','Mot.desviac.','Texto notific.'])
fHh.columns=['NPersonal','nombre','Orden','Operacion','TrabajoReal']

oslist = list(fQplus.index.values)
fHh = addOSMadreIndex(fHh, dropOrden=False, ordenName = 'Orden', osFilter=oslist,index=False)
fHh = fHh.groupby(['OSMadre','Orden','Operacion','NPersonal'])['TrabajoReal'].sum().reset_index()
fHh = fHh[fHh.TrabajoReal!=0].reset_index(drop=True)

# Filtrar por los trabajadores operacionales como los mecanicos y tecnicos
listaPersonal = list(fTrabajadores.NPersonal)
fHh = fHh[fHh['NPersonal'].isin(listaPersonal)]

## Operaciones

In [22]:
fOperaciones = hOperaciones.copy()
fOperaciones = fOperaciones.drop(columns=['ClAct','Ctrl','Status de sistema','Clv.mod.'])

fOperaciones.columns = ['serieComp','Orden','Operacion','Descripcion','serieEquipo','TrabajoCotizado','TrabajoReal']

oslist = list(fQplus.index.values)
fOperaciones = addOSMadreIndex(fOperaciones,dropOrden=False,ordenName='Orden',osFilter=oslist,index=False)

# Considera solo aquellas operaciones con registro en el trabajo real
fOperaciones = fOperaciones[(fOperaciones.TrabajoReal!=0) | (fOperaciones.TrabajoCotizado!=0)].dropna().reset_index(drop=True)

## WIP

In [23]:
fWip = hWip.copy()
fWip.drop(columns=['Aviso','Cl.Aviso','SIMS','Centro planificación','Area Emp','Sociedad','Clase Orden','Status de sistema','Apr Hs','NF','SolicParts','PrtsEntreg','BO','TTA','Norm.Liq.','Garant','N° Claim','Cliente','Vendedor','Tipo Equipo','Fabricante','N° Cotiz','Stat Claim','Precio KMT','Precio Misc','Precio Viaje','Costo Misc','Costo KMT','Fe Prom Ej','Fe AP OS','Dias Inac','Fe Cotiz',	'Fe Ent Co','Fe Aprob',	'Fe Prom Ej','Fe ConcS','Fe CieTec','Equipo','Ord. Liqui','TAM','Status Usu.Orden de Servicio','Avance %','TotHsRe','TotHsPl','Precio Repuestos','Precio Mano de Obra','Precio TTA','Precio total de venta','Serie','Modelo'], inplace=True)
fWip.columns = ['OSMadre','Orden','costoRepuesto','costoMO','costoTTA','costoTotal','wip']
fWip = fWip[fWip.OSMadre.isin(oslist)]

## Cursos

### Parcial

In [24]:
fHistorial = hHistorial[['ID de empleado','Estado Finalizado','Calificación','Puntaje','ID de referencia de la oferta de cursos']]

fHistorial.columns = ['NPersonal','estadoFinalizado','calificacion','puntaje','IDCurso']

### Completo (Capstone Previo)

In [25]:
fCursosAll = hCursosAll.copy()
fCursosAll

,IDCurso,Habilidades,Desarme,Tipo,habilidadesHistorial,Nivel
0,HME0924,Embalaje de Componentes con Plástico Termo Ret...,0,Power Train,Embalaje de Componentes con Plástico Termo Ret...,M1
1,HME0646,Uso de Llaves Dinamométricas e Indicador de Gr...,0,Power Train,Uso de Llaves Dinamométricas e Indicador de Gr...,M1
2,HME1108,Uso de Llave de Impacto Neumática,0,Power Train,Uso de Llave de Impacto Neumática,M1
3,HME1201,Uso Correcto Instrumentación CRC (Pie de Metro...,0,Power Train,Uso Correcto Instrumentación CRC (Pie de Metro...,M1
4,HME2050,Lavadoras Recirculatorias.,0,Power Train,Lavadoras Recirculatorias.,M1
...,...,...,...,...,...,...
123,HME4000,Conocimientos Avanzados del WPMENU en DBS,0,Power Train,-,S1
124,HME4002,Reclamos de PIP y PSP,0,Power Train,Reclamos de PIP y PSP,S1
125,HME4004,Análisis de Información Descargada de Sistema ...,0,Power Train,Análisis de información descargada del Sistema...,S1
126,HME4011,Programación de Taller,0,Power Train,Programación de taller,S1


# Exportar Datos

In [26]:
ruta_destino = '03_Preprocesamiento/Datos'

# De SAP
exportar_csv(fRepuestos,ruta_destino,'fRepuestos',False)
exportar_csv(fHh,ruta_destino,'fHh',False)
exportar_csv(fOperaciones,ruta_destino,'fOperaciones',False)

# De Qplus
exportar_csv(aQplus,ruta_destino,'aQplus',False)
exportar_csv(fQplus,ruta_destino,'fQplus',True)

# De Trabajadores
exportar_csv(fTrabajadores,ruta_destino,'fTrabajadores',False)

# De Wip
exportar_csv(fWip,ruta_destino,'fWip',False)

# De Cursos
exportar_csv(hCursos,ruta_destino,'hCursos',False)
exportar_csv(hCursosAll,ruta_destino,'hCursosAll',False)
exportar_csv(fHistorial,ruta_destino,'fHistorial',False)